In [1]:
import tensorflow as tf
import gym
import pickle
import numpy as np
import os

In [2]:
import sys
sys.path.insert(0, '/data/apps/roboschool/agent_zoo/')

from RoboschoolHalfCheetah_v1_2017jul import ZooPolicyTensorflow as ZooPolicyTensorflow_RoboschoolHalfCheetah_v1
from RoboschoolHumanoid_v1_2017jul import ZooPolicyTensorflow as ZooPolicyTensorflow_RoboschoolHumanoid_v1

In [3]:
rollouts_dir = 'expert_data'
rollouts_num = 1000
# env_name = "RoboschoolHalfCheetah-v1"
env_name = "RoboschoolHumanoid-v1"

In [4]:
roll_out_file = os.path.join(rollouts_dir, env_name + '_dagger_2.pkl')
result_file = os.path.join(rollouts_dir, env_name + '_results.pkl')
dagger_file = os.path.join(rollouts_dir, env_name + '_dagger_3.pkl')

In [5]:
# read roll-outs
with open(roll_out_file, 'rb') as f:
    roll_outs = pickle.loads(f.read())

In [6]:
print(roll_outs['observations'].shape)
print(roll_outs['actions'].shape)

(1240369, 44)
(1240369, 17)


In [7]:
# read results
with open(result_file, 'rb') as f:
    results = pickle.loads(f.read())

In [8]:
print(results['observations'].shape)

(742047, 44)


In [9]:
sess = tf.InteractiveSession()
env = gym.make(env_name)
if (env_name == "RoboschoolHalfCheetah-v1"):
    ZooPolicyTensorflow = ZooPolicyTensorflow_RoboschoolHalfCheetah_v1
elif (env_name == "RoboschoolHumanoid-v1"):
    ZooPolicyTensorflow = ZooPolicyTensorflow_RoboschoolHumanoid_v1
pi = ZooPolicyTensorflow("mymodel1", env.observation_space, env.action_space)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: Environment '<class 'roboschool.gym_mujoco_walkers.RoboschoolHumanoid'>' has deprecated methods '_step' and '_reset' rather than 'step' and 'reset'. Compatibility code invoked. Set _gym_disable_underscore_compat = True to disable this behavior.


/data/apps/anaconda3/envs/cs294/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [10]:
actions = []
for obs in results['observations']:
    actions.append(pi.act(obs, env))
env.close()

In [11]:
expert_data = {'observations': np.concatenate((roll_outs['observations'], results['observations'])),
               'actions': np.concatenate((roll_outs['actions'], np.array(actions)))}
with open(dagger_file, 'wb') as f:
    pickle.dump(expert_data, f, pickle.HIGHEST_PROTOCOL)

In [12]:
np.concatenate((roll_outs['observations'], results['observations'])).shape

(1982416, 44)